In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import os
import email
import email.policy
from bs4 import BeautifulSoup

os.listdir('../input/ham-and-spam-dataset/')
os.listdir('../input/ham-and-spam-dataset/')

In [ ]:
ham_filenames = [name for name in sorted(os.listdir('../input/ham-and-spam-dataset/ham')) ]
spam_filenames = [name for name in sorted(os.listdir('../input/ham-and-spam-dataset/spam')) ]
plt.bar([0],[len(ham_filenames),], align='center', alpha=0.5,)
plt.bar([1],len(spam_filenames), align='center', alpha=0.5)
plt.legend(['ham','spam'])
plt.ylabel('No of mailes')
plt.title('No of mails Analysis')
plt.show()

In [ ]:
def load_email(is_spam, filename):
    directory = "../input/ham-and-spam-dataset/spam" if is_spam else "../input/ham-and-spam-dataset/ham"
    with open(os.path.join(directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
    
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [ ]:
testEmail = ham_emails[11]
testEmailContent = testEmail.get_content()
print(testEmailContent)

In [ ]:
from collections import Counter

def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

ham_structure = structures_counter(ham_emails)
spam_structure = structures_counter(spam_emails)
for email in spam_emails:
    if get_email_structure(email) == 'text/html':
        testEmail = email
        break
def html_to_plain(email):
    try:
        soup = BeautifulSoup(email.get_content(), 'html.parser')
        return soup.text.replace('\n\n','')
    except:
        return "empty"

def email_to_plain(email):
    struct = get_email_structure(email)
    for part in email.walk():
        partContentType = part.get_content_type()
        if partContentType not in ['text/plain','text/html']:
            continue
        try:
            partContent = part.get_content()
        except: # in case of encoding issues
            partContent = str(part.get_payload())
        if partContentType == 'text/plain':
            return partContent
        else:
            return html_to_plain(part)



In [ ]:
orig=[]
h=[]
s=[]
import pandas as pd 
import re
count=0
for text in ham_emails:
    
   
    text=email_to_plain(text)
        
    try:
            
        text = text.replace('.','')
        text = text.replace(':','')
        text = text.replace(',','')
        text = text.replace('!','')
        text = text.replace('?','')
        text=text.replace('\n','')
        text=text.replace('\t','')
        h.append(text)

    except:
       pass
    
   
#label=[1]*len(ham_emails)

for text in spam_emails:
   
        text=email_to_plain(text)

        try:
            
            text = text.replace('.','')
            text = text.replace(':','')
            text = text.replace(',','')
            text = text.replace('!','')
            text = text.replace('?','')
            text = text.replace('\n','')
            text=text.replace('\t','')
            s.append(text)

        except:
            pass
        
    

In [ ]:
text

In [ ]:
label=[0]*len(h)+[1]*len(s)

In [ ]:
texts = h+s # Extract text
target = label # Extract target

In [ ]:
texts[0]

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
wl=50000
tokenizer=Tokenizer(wl,filters='!@#$%^&*{}]~`[',lower=True)
tokenizer.fit_on_texts(texts)
text=tokenizer.texts_to_sequences(texts)


In [ ]:
len(text[9])

In [ ]:
msl=250
text=pad_sequences(text,msl)

In [ ]:
from keras.layers import Embedding, LSTM,Dense,Dropout,Dense
from keras.models import Sequential

In [ ]:
 help(Embedding)

In [ ]:
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
y=to_categorical(target)
x_train,x_test,y_train,y_test=train_test_split(text,y,test_size=0.3, random_state=40)

In [ ]:
len(x_train[2])

In [ ]:
model=Sequential()
model.add(Embedding(50000,64,input_length=250))
model.add(LSTM(64,input_shape=(250,10),activation='tanh'))
model.add(Dense(10,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(x_train,y_train,epochs=10, validation_split=0.2 ,batch_size=10)

In [ ]:
acc=model.evaluate(x_test,y_test)


In [ ]:
new_complaint = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=250)
pred = model.predict(padded)
print(pred)

In [ ]:
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import matplotlib.pyplot as plt
hist=history
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']
xc=range(10)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
y_pred=model.predict(x_test)
y_test
labels=[1,0]
y_test_label=[]
y_pred_label=[]

for i in y_test:
   y_test_label.append(labels[np.argmax(i)])
for i in y_pred:
    y_pred_label.append(labels[np.argmax(i)])

In [ ]:
y_pred_label

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics 
data = metrics.confusion_matrix(y_test_label, y_pred_label)
import seaborn as sb
sb.heatmap(data, annot=True)

**Data Prediction**

In [ ]:


new_complaint = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']

seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=250)
pred = model.predict(padded)
labels = ['ham','spam']
print( labels[np.argmax(pred)])

In [ ]:
new_complaint = ['todays weather is cool ,isnt it']

seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=250)
pred = model.predict(padded)
labels = ['ham','spam']
print( labels[np.argmax(pred)])